Imports and utility functions for use on MacBook. This doesn't import packages bcolz and psycopg2 which I haven't been able to install on Mac.

Also, decided not to import rpy2 for now as giving warnings

Use in other notebooks, e.g.:

```
%run imports_mac.ipynb
```

This was copied from http://github.com/malariagen/ag1000g/tree/master/notebooks/aliman

In [1]:
import os
if 'docker_image' in os.environ:
    print('docker image:', os.environ['docker_image'])

## Notebook customisation

In [2]:
# %%HTML
# <style type="text/css">
# .container {
#     width: 100%;
# }
# #maintoolbar {
#     display: none;
# }
# #header-container {
#     display: none;
# }
# #notebook {
#     padding-top: 0;
# }
# </style>

## Imports

In [3]:
# python standard library
import sys
import os
import operator
import itertools
import collections
import functools
import glob
import csv
import datetime
import bisect
import sqlite3
import subprocess
import random
import gc
import shutil
import shelve
import contextlib
import tempfile
import math

In [4]:
# general purpose third party packages

import cython
%reload_ext Cython

import numpy as np
nnz = np.count_nonzero
import scipy
import scipy.stats
import scipy.spatial.distance
import numexpr
import h5py
import tables
# import bcolz
import dask
import dask.array as da
import pandas
import IPython
from IPython.display import clear_output, display, HTML
# import rpy2
# import rpy2.robjects as ro
# %reload_ext rpy2.ipython
import statsmodels
import sklearn
import sklearn.decomposition
import sklearn.manifold
import sh
import sqlalchemy
import pymysql
# import psycopg2
import petl as etl
etl.config.display_index_header = True
import humanize
from humanize import naturalsize, intcomma, intword

In [8]:
import warnings

def showwarning(message, category, filename, lineno, file=None, line=None):
    # ignore this one from matplotlib
    if str(message).startswith('axes.color_cycle'):
        return
#     warnings.formatwarning(message, category, filename, lineno, file=file, line=line)
    warnings.formatwarning(message, category, filename, lineno, line=line)

warnings.showwarning = showwarning

In [6]:
# plotting setup
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
import matplotlib_venn as venn
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')
sns.set_style('ticks')
rcParams = plt.rcParams
rcParams['font.size'] = 8
rcParams['axes.labelsize'] = 8
rcParams['xtick.labelsize'] = 8
rcParams['ytick.labelsize'] = 8
rcParams['legend.fontsize'] = 8
rcParams['axes.linewidth'] = .5
rcParams['lines.linewidth'] = .5
rcParams['patch.linewidth'] = .5
# rcParams['font.family'] = 'arial'
rcParams['ytick.direction'] = 'out'
rcParams['xtick.direction'] = 'out'
rcParams['savefig.jpeg_quality'] = 100
rcParams['savefig.dpi'] = 120
rcParams['lines.markeredgewidth'] = .5
rcParams['figure.figsize'] = (4.85, 3)

In [9]:
# bio third party packages
import Bio
import pyfasta
import pysam
# currently broken
# import pysamstats
import petlx
import petlx.bio
import vcf
# import vcfnp
# import anhima
import allel
try:
    import ete3
except ImportError:
    pass

## Utility functions

In [10]:
_slog_indent = -2

def log(*msg):
    s = ' '.join(map(str, msg))
    print(s, file=sys.stdout)
    sys.stdout.flush()


@contextlib.contextmanager
def timer(*msg):
    before = datetime.datetime.now()
    try:
        yield
    except:
        after = datetime.datetime.now()
        elapsed = (after - before).total_seconds()
        done = 'errored after %s' % humanize.naturaldelta(elapsed)
        if not msg:
            msg = done
        else:
            msg = ', '.join(map(str, msg)) + ', ' + done
        print(msg, file=sys.stderr)
        sys.stderr.flush()   
        raise
    else:
        after = datetime.datetime.now()
        elapsed = (after - before).total_seconds()
        done = 'done in %s' % humanize.naturaldelta(elapsed)
        if not msg:
            msg = done
        else:
            msg = ', '.join(map(str, msg)) + ', ' + done
        print(msg, file=sys.stdout)
        sys.stdout.flush()
        
        
@contextlib.contextmanager
def section(*title):
    global _slog_indent
    before = datetime.datetime.now()
    _slog_indent += 2                  
    prefix = (' ' * _slog_indent) + '[' + ', '.join(map(str, title)) + '] '
    
    def slog(*msg, file=sys.stdout):
        print(prefix + ' '.join(map(str, msg)), file=file)
        file.flush()
    
    slog('begin')
                            
    try:
        yield slog
    
    except:
        after = datetime.datetime.now()
        elapsed = (after - before).total_seconds()
        msg = 'errored after %s' % humanize.naturaldelta(elapsed)
        slog(msg, file=sys.stderr)
        _slog_indent -= 2                  
        raise
    
    else:
        after = datetime.datetime.now()
        elapsed = (after - before).total_seconds()
        msg = 'done in %s' % humanize.naturaldelta(elapsed)
        slog(msg, file=sys.stdout)
        _slog_indent -= 2                  
    

In [11]:
# autosomes = '2R', '2L', '3R', '3L'
# chromosomes = autosomes + ('X',)


# class GenomeFigure(object):
    
#     def __init__(self, genome, *args, **kwargs):
#         self.chromosomes = kwargs.pop('chromosomes', ['2R', '2L', '3R', '3L', 'X'])
#         maxchrsize = max(np.array(genome[chrom]).size for chrom in self.chromosomes)
#         fig = plt.figure(*args, **kwargs)
#         self.fig = fig
#         self.ax = dict()
#         for i, chrom in enumerate(self.chromosomes):
#             ax = fig.add_subplot(3, 2, i+1)
#             self.ax[chrom] = ax
#             S = np.array(genome[chrom])
#             if i % 2 == 1:
#                 sns.despine(ax=ax, offset=10, top=True, left=True, right=False)
#                 ax.set_xlim(0, maxchrsize)
#                 ax.yaxis.tick_right()
#                 ax.yaxis.set_label_position('right')
#             else:
#                 ax.set_xlim((S.size)-(maxchrsize), S.size)
#                 ax.yaxis.tick_left()
#                 sns.despine(ax=ax, offset=10, top=True, left=False, right=True)
#             ax.set_xticks(range(0, S.size, int(5e6)))
#             ax.set_xticklabels(range(0, int(S.size/1e6), 5))
#             ax.set_title(chrom, fontweight='bold')
#             ax.xaxis.tick_bottom()
#         fig.tight_layout()
        
#     def apply(self, f, **kwargs):
#         chromosomes = kwargs.pop('chromosomes', self.chromosomes)
#         for chrom in chromosomes:
#             ax = self.ax[chrom]
#             f(chrom, ax, **kwargs)

            
def subplots(*args, despine=None, **kwargs):
    fig, ax = plt.subplots(*args, **kwargs)
    if despine is None:
        despine = dict(offset=10, trim=False)
    if hasattr(ax, '__len__'):
        for a in ax:
            sns.despine(ax=a, **despine)
    else:
        sns.despine(ax=ax, **despine)
    return fig, ax


In [12]:
def pip_list():
    import pip
    cmd = pip.commands.ListCommand()
    cmd.main([])

In [13]:
def str_ascii(b):
    if isinstance(b, (tuple, list)):
        return type(b)([str(i, 'ascii') for i in b])
    elif isinstance(b, np.ndarray):
        return np.array([str(i, 'ascii') for i in b.flatten()]).reshape(b.shape)
    else:
        return str(b, 'ascii')
        